In [1]:
import pandas as pd
import numpy as np

## EPH survey data

In [19]:
df = pd.read_csv('./../resumen_EPH_hogar.csv')
df = df.loc[df.IV1 != 9]
df['IV10'] = df['IV10'].map({1: 1, 2: 2, 3: 2, 0: 0, 9: 9})
df['II9'] = df['II9'].map({1: 1, 2: 2, 3: 2, 4: 4, 0: 0})
df['II7'] = df['II7'].map({1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 6, 8: 6, 9: 6, 0: 0})
df['IX_TOT'] = df['IX_TOT'].clip(0, 8)


In [4]:
# df.groupby(['AGLOMERADO']).nunique()

In [3]:
# for col in df.iloc[:, 2:].columns:
#     print col
#     print sorted(df[col].unique())

## 2010 ARG Census data

In [3]:
path = './../../Datos_censo/Preguntas/'

VIVIENDA_V01 = pd.read_csv(path + 'VIVIENDA-V01.csv', encoding = 'latin-1')
HOGAR_H05 = pd.read_csv(path + 'HOGAR-H05.csv', encoding = 'latin-1') # Pisos
HOGAR_H06 = pd.read_csv(path + 'HOGAR-H06.csv', encoding = 'latin-1') # Cubierta techos
HOGAR_H07 = pd.read_csv(path + 'HOGAR-H07.csv', encoding = 'latin-1') # Cielorraso
HOGAR_H08 = pd.read_csv(path + 'HOGAR-H08.csv', encoding = 'latin-1') # Agua
HOGAR_H09 = pd.read_csv(path + 'HOGAR-H09.csv', encoding = 'latin-1') # Procedencia agua
HOGAR_H10 = pd.read_csv(path + 'HOGAR-H10.csv', encoding = 'latin-1') # Tiene baño
HOGAR_H11 = pd.read_csv(path + 'HOGAR-H11.csv', encoding = 'latin-1') # Baño con cadena
HOGAR_H12 = pd.read_csv(path + 'HOGAR-H12.csv', encoding = 'latin-1') # Desague baño
HOGAR_H13 = pd.read_csv(path + 'HOGAR-H13.csv', encoding = 'latin-1') # Baño uso exclusivo
HOGAR_H14 = pd.read_csv(path + 'HOGAR-H14.csv', encoding = 'latin-1') # Combustible cocina
HOGAR_PROP = pd.read_csv(path + 'HOGAR-PROP.csv', encoding = 'latin-1') # Regimen tenencia
HOGAR_TOTPERS = pd.read_csv(path + 'HOGAR-TOTPERS.csv', encoding = 'latin-1') # Personas en hogar

df_list = [VIVIENDA_V01, HOGAR_H05, HOGAR_H06, HOGAR_H07, HOGAR_H08, HOGAR_H09, 
          HOGAR_H10, HOGAR_H11, HOGAR_H12, HOGAR_H13, HOGAR_H14, HOGAR_PROP, HOGAR_TOTPERS]

## Harmonise variables (let valid answers match exactly in both datasets)

In [4]:

VIVIENDA_V01['2 Departamento'] = VIVIENDA_V01['4 Departamento']
VIVIENDA_V01['3 Pieza de inquilinato'] = VIVIENDA_V01['5 Pieza en inquilinato']
VIVIENDA_V01[u'4 Pieza en hotel / pensión'] = VIVIENDA_V01[u'6 Pieza en hotel familiar o pensión']
VIVIENDA_V01[u'5 Local no construido para habitación'] = VIVIENDA_V01[u'7 Local no construido para habitación']
VIVIENDA_V01['6 Otros'] = VIVIENDA_V01['2 Rancho'] + VIVIENDA_V01['3 Casilla'] + VIVIENDA_V01[u'8 Vivienda móvil'] + VIVIENDA_V01['9 Persona/s viviendo en la calle']

IV1_censo = VIVIENDA_V01[['radio', '1 Casa', '2 Departamento', '3 Pieza de inquilinato', u'4 Pieza en hotel / pensión',
                         u'5 Local no construido para habitación', '6 Otros', 'TOTAL']]
IV3_censo = HOGAR_H05
IV3_censo.columns = ['radio', '1 Mosaico / baldosa / madera / cerámica / alfombra', 
                    '2 Cemento / ladrillo fijo', '3 Ladrillo suelto / tierra', '4 Otro', 'TOTAL']

IV4_censo = HOGAR_H06
IV4_censo.columns = ['radio', '1 Membrana / cubierta asfáltica', '2 Baldosa / losa sin cubierta', '3 Pizarra / teja', 
                     '4 Chapa de metal sin cubierta', '5 Chapa de fibrocemento / plástico', '6 Chapa de cartón', 
                     '7 Caña / tabla / paja con barro / paja sola', '9 N/S. Depto en propiedad horizontal', 'TOTAL']

IV5_censo = HOGAR_H07
IV6_censo = HOGAR_H08
IV7_censo = HOGAR_H09
IV7_censo['4 Otra fuente'] = HOGAR_H09['4 Pozo'] + HOGAR_H09['5 Transporte por cisterna'] + \
 HOGAR_H09[u'6 Agua de lluvia, río, canal, arroyo o acequia']
IV7_censo = IV7_censo[[u'radio', u'1 Red pública', u'2 Perforación con bomba a motor',
                      u'3 Perforación con bomba manual', u'4 Otra fuente', u'TOTAL']]

IV8_censo = HOGAR_H10
IV10_censo = HOGAR_H11
IV11_censo = HOGAR_H12
II9_censo = HOGAR_H13

II8_censo = HOGAR_H14
II8_censo['2 Gas de tubo / garrafa'] = HOGAR_H14['3 Gas en tubo'] + HOGAR_H14['4 Gas en garrafa']
II8_censo[u'3 Kerosene / leña/ carbón'] = HOGAR_H14[u'6 Leña o carbón']
II8_censo['4 Otro'] = HOGAR_H14[u'2 Gas a granel (zeppelin)'] + HOGAR_H14[u'5 Electricidad'] + HOGAR_H14[u'7 Otro']

II8_censo = II8_censo[['radio', '1 Gas de red', '2 Gas de tubo / garrafa', u'3 Kerosene / leña/ carbón', '4 Otro', 'TOTAL']]

II7_censo = HOGAR_PROP

IX_TOT_censo = HOGAR_TOTPERS

## Intento Max Ent

#### Margins

In [551]:
# From whole country data, we can only have margins. Same for a given district or precint.

tot_viviendas = 12171675.
IV5_censo.iloc[:, 1:].sum()/tot_viviendas

1 Sí     0.723948
2 No     0.276052
TOTAL    1.000000
dtype: float64

In [214]:
IV3_censo.iloc[:, 1:].sum()/tot_viviendas

1 Mosaico / baldosa / madera / cerámica / alfombra    0.765829
2 Cemento / ladrillo fijo                             0.202845
3 Ladrillo suelto / tierra                            0.025842
4 Otro                                                0.005484
TOTAL                                                 1.000000
dtype: float64

In [231]:
IV6_censo.iloc[:, 1:].sum()/tot_viviendas

1 Por cañeria dentro de la vivienda               0.885473
2 Fuera de la vivienda pero dentro del terreno    0.092493
3 Fuera del terreno                               0.022033
TOTAL                                             1.000000
dtype: float64

#### Prior probabilities (matrix entries) from survey info

In [603]:
qjkl = df.groupby(['IV4', 'IV3', 'IV6'])[['CODUSU']].nunique() # how many IDs with each combination
qjkl /= qjkl.sum()

q_ix = pd.MultiIndex.from_product(qjkl.index.levels, names=qjkl.index.names)
qjkl = qjkl.reindex(q_ix, fill_value = 0)

qjkl = qjkl.unstack([2, 1]).sort_index(axis = 1)

info_dfs = [IV4_censo, IV6_censo, IV3_censo]
T = len(info_dfs)

margins_s = []
margins_v = []
shape_ = np.zeros(T, dtype = int)

for t in range(T):
    margin_series = (info_dfs[t].iloc[:, 1:-1].sum()/info_dfs[t].TOTAL.sum())
    margins_s += [margin_series]
    margins_v += [margin_series.values]
    shape_[t] = len(margin_series)
    
a, b, c = tuple([m.values for m in margins_s])
df_ = pd.DataFrame(np.prod(np.ix_(a, b, c)).reshape(qjkl.values.shape), 
                  columns = qjkl.columns, index = qjkl.index)
uncorr_fill_p = df_

uncorr_fill_q = df_ * (qjkl == 0).astype(int)
qjkl = (1 - uncorr_fill_q.sum().sum())*qjkl + uncorr_fill_q

In [554]:
l = np.prod(shape_)
cube = np.arange(l).reshape(shape_)
# cube[:, 2, :].flatten()

In [431]:
for t in range(T):
    cube_tr = cube.transpose(tuple(np.roll(np.arange(T), - (t))))

In [163]:
slic.flatten()

array([ 3,  7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63, 67,
       71, 75, 79, 83, 87, 91, 95])

In [432]:
p_0 = q_mat.flatten()

In [263]:
constraints = []
for dim in range(T):
    cube_tr = cube.transpose(tuple(np.roll(np.arange(T), - (dim))))
    ix = cube_tr[i].flatten()
    for j in range(shape_[dim]):
        ix = cube_tr[j].flatten()
        marcond = margins[dim][j]
        def f(p, ix = ix):
            return p[ix].sum() - marcond
        
        
#     for j in shape_[dim]: # j in (0, J), k in (0, K), etc.
#         def f(p, dim = dim):
#             return p_mat.sum(axis = tuple([t for t in range(len(shape)) if t != dim])) - margins[dim].values
        constraints.append(f)
cons = [{'type':'eq', 'fun': con} for con in constraints]

<function f at 0x7f4ad8de5ed8>
<function f at 0x7f4ad8de5c80>
<function f at 0x7f4ad8de5b90>
<function f at 0x7f4ad8de5c08>
<function f at 0x7f4ad8de5f50>
<function f at 0x7f4ae006f5f0>
<function f at 0x7f4ae006f578>
<function f at 0x7f4ae006f6e0>
<function f at 0x7f4ae006f668>
<function f at 0x7f4ae006f320>
<function f at 0x7f4ae006f500>
<function f at 0x7f4ae006faa0>
<function f at 0x7f4ae006f398>
<function f at 0x7f4ae006f2a8>
<function f at 0x7f4ae006f488>


In [433]:
# M_0 = np.array([[1, 0.1], [0.1, 1]])

cube_tr = cube.transpose(tuple(np.roll(np.arange(T), - (dim))))
# ix = cube_tr[i].flatten()
    
print dim
constraints = []
for j in range(shape_[dim]):
    marconds = margins_v[dim]
    def f(p, cube_tr = cube_tr, marg = marg, j = j):
        return p[cube_tr[j].flatten()].sum() - marconds[j]
    constraints.append(f)
    
cons = [{'type':'eq', 'fun': con} for con in constraints]

def MaxEnt(p_mat, q_mat = q_mat):
    p_mat = np.array(p_mat).reshape(q_mat.shape)
    return (p_mat*np.log(p_mat/q_mat)).sum()
    
res = minimize(lambda M: MaxEnt(M), q_mat, constraints = cons)
res['x'].reshape(q_mat.shape)

1


/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:18: RuntimeWarning: invalid value encountered in log


array([[[  2.39420239e-01,   1.25704273e-02,   8.12146613e-04,
           5.53879338e-04],
        [  3.18155942e-03,   4.09274079e-03,   2.21958013e-04,
           3.79529481e-04],
        [  2.70973492e-04,   6.74479942e-05,   4.70843956e-04,
           9.60623093e-05]],

       [[  1.88981967e-01,   1.97959212e-02,   3.48031954e-04,
           3.07030977e-04],
        [  5.75843454e-03,   9.38602774e-03,   4.78377923e-04,
           6.33436412e-04],
        [  2.57279000e-04,   1.32994602e-03,   8.48609324e-05,
           1.78678788e-04]],

       [[  6.33243338e-02,   1.80699211e-03,   6.28310010e-05,
           1.15463593e-04],
        [  3.12105169e-04,   5.50747703e-04,   8.33212021e-04,
           1.69962807e-04],
        [  7.85391094e-03,   1.02622849e-03,   3.06232807e-04,
           6.57841655e-05]],

       [[  2.22934582e-01,   5.10439201e-02,   7.60562970e-04,
           4.50599532e-04],
        [  1.08009894e-02,   4.14838298e-02,   3.33316007e-03,
           1.61128830

In [379]:
ix
cube_tr[j].flatten()

array([ 3,  7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63, 67,
       71, 75, 79, 83, 87, 91, 95])

In [393]:
np.arange(32).reshape(8, 4)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31]])

In [507]:
q_mat[0]

array([[  2.65412417e-01,   1.28106749e-02,   8.10802208e-04,
          7.07609200e-04],
       [  8.55027783e-04,   1.12038123e-03,   5.89674333e-05,
          9.09375621e-05],
       [  4.42255750e-05,   1.47418583e-05,   1.02083567e-04,
          2.16627937e-05]])

In [525]:
M_0 = q_mat + 0.01

constraints = []

shape_ = M_0.shape
l = np.prod(shape_)
cube = np.arange(l).reshape(shape_)

for dim in range(1, T):
    cube_tr = cube.transpose(tuple(np.roll(np.arange(T), - (dim)))) 
    print cube_tr.shape
    marg = margins_v[dim]
    print marg

    for j in range(shape_[dim]):
        print j
        def f(M, cube_tr = cube_tr, marg = marg, j = j):
            return M[cube_tr[j]].sum() - marg[j]

        constraints.append(f)
    
cons = [{'type':'eq', 'fun': con} for con in constraints]

def MaxEnt(p_mat, q_mat = M_0):
    p_mat = np.array(p_mat).reshape(q_mat.shape)
    return (p_mat*np.log(p_mat/q_mat)).sum()
    
res = minimize(lambda M: MaxEnt(M), M_0, constraints = cons)
# res['x'].reshape(M_0.shape)
print res['x'][cube_tr[j]].sum() - marg[j]

(3, 4, 8)
[ 0.88547345  0.09249319  0.02203337]
0
1
2
(4, 8, 3)
[ 0.76582935  0.20284505  0.0258418   0.0054838 ]
0
1
2
3
0.0352245665055


/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:26: RuntimeWarning: invalid value encountered in log


In [594]:
M_0 = qjkl['CODUSU'].values + 0.01

cons = []

shape_ = (2, 3, 4)
l = np.prod(shape_)
cube = np.arange(l).reshape(shape_)
for dim in [0, 1, 2]:
    cube_tr = cube.transpose(tuple(np.roll(np.arange(T), - (dim)))) 
    print cube_tr.shape
    marg = margins_v[dim]
    print marg
    
    def cons_j(M, j):
        return M[cube_tr[j]].sum() - marg[j]
    
    cons_per_j = [{'type':'eq', 'fun': lambda M, j=j: cons_j(M, j)} for j in np.arange(shape_[dim])]

#     for j in range(shape_[dim]):
#         print j
#         def f(M, cube_tr = cube_tr, marg = marg, j = j):
#             return M[cube_tr[j]].sum() - marg[j]

    cons += cons_per_j
        
def f_norm(M, cube_tr = cube_tr, marg = marg, j = j):
            return M[cube_tr[j]].sum() - marg[j]
    
# cons = [{'type':'eq', 'fun': con} for con in constraints]

def MaxEnt(p, q = M_0.flatten()):
#     p_mat = np.array(p_mat).reshape(q_mat.shape)
    return (p*np.log(p/q)).sum()
    
res = minimize(lambda M: MaxEnt(M), M_0, constraints = cons)
res['x']


(2, 3, 4)
[ 0.72394753  0.27605247]
(3, 4, 2)
[ 0.88547345  0.09249319  0.02203337]
(4, 2, 3)
[ 0.76582935  0.20284505  0.0258418   0.0054838 ]


array([ 0.82082022,  0.06215215,  0.01100549,  0.01201097,  0.01344527,
        0.01433246,  0.01017744,  0.01002957,  0.01026616,  0.01023658,
        0.0104122 ,  0.01008747,  0.07256187,  0.0550546 ,  0.01125686,
        0.01031052,  0.01241021,  0.02064631,  0.01162652,  0.01005915,
        0.01016265,  0.01068018,  0.0102218 ,  0.01003335])

In [595]:
print res['x'].reshape(2, 3, 4).sum(axis = (1,2))
print res['x'].reshape(2, 3, 4).sum(axis = (0,2))
print res['x'].reshape(2, 3, 4).sum(axis = (0,1))

[ 0.99497598  0.24502402]
[ 1.05517268  0.10272692  0.0821004 ]
[ 0.93966638  0.17310229  0.0647003   0.06253103]


In [596]:
print [margins_v[dim] for dim in range(T)]

[array([ 0.72394753,  0.27605247]), array([ 0.88547345,  0.09249319,  0.02203337]), array([ 0.76582935,  0.20284505,  0.0258418 ,  0.0054838 ])]


In [597]:
M_0 = qjkl['CODUSU'][1].values + 0.01

constraints = []

shape_ = M_0.shape
l = np.prod(shape_)
cube = np.arange(l).reshape(shape_)
for dim in [0, 1]:
    cube_tr = cube.transpose(tuple(np.roll(np.arange(2), - (dim)))) 
    print cube_tr.shape
    if dim == 0:
        marg = margins_v[0]
    elif dim == 1:
        marg = margins_v[2]

    print marg
    for j in range(shape_[dim]):
        def f(M, cube_tr = cube_tr, marg = marg, j = j):
            return M[cube_tr[j]].sum() - marg[j]

        constraints.append(f)
    
cons = [{'type':'eq', 'fun': con} for con in constraints]

def MaxEnt(p, q = M_0.flatten()):
#     p_mat = np.array(p_mat).reshape(q_mat.shape)
    return (p*np.log(p/q)).sum()
    
res = minimize(lambda M: MaxEnt(M), M_0, constraints = cons)
res['x'].round(3).reshape(M_0.shape)

(2, 4)
[ 0.72394753  0.27605247]
(4, 2)
[ 0.76582935  0.20284505  0.0258418   0.0054838 ]


array([[ 0.821,  0.062,  0.011,  0.012],
       [ 0.073,  0.055,  0.011,  0.01 ]])

In [588]:
cons

[{'fun': <function __main__.f>, 'type': 'eq'},
 {'fun': <function __main__.f>, 'type': 'eq'},
 {'fun': <function __main__.f>, 'type': 'eq'},
 {'fun': <function __main__.f>, 'type': 'eq'},
 {'fun': <function __main__.f>, 'type': 'eq'},
 {'fun': <function __main__.f>, 'type': 'eq'}]

In [592]:
constraints

[{'fun': <function __main__.<lambda>>, 'type': 'eq'},
 {'fun': <function __main__.<lambda>>, 'type': 'eq'},
 {'fun': <function __main__.<lambda>>, 'type': 'eq'},
 {'fun': <function __main__.<lambda>>, 'type': 'eq'},
 {'fun': <function __main__.<lambda>>, 'type': 'eq'},
 {'fun': <function __main__.<lambda>>, 'type': 'eq'},
 {'fun': <function __main__.<lambda>>, 'type': 'eq'},
 {'fun': <function __main__.<lambda>>, 'type': 'eq'},
 {'fun': <function __main__.<lambda>>, 'type': 'eq'}]

In [560]:
pd.DataFrame(res['x'].reshape(2, 4)).sum(axis = 0)

0    0.893382
1    0.117207
2    0.022262
3    0.022321
dtype: float64

In [561]:
pd.DataFrame(res['x'].reshape(2, 4)).sum(axis = 1)

0    0.905989
1    0.149184
dtype: float64

In [566]:
res['x']

array([ 0.82082022,  0.06215215,  0.01100549,  0.01201097,  0.07256187,
        0.0550546 ,  0.01125686,  0.01031052])

In [569]:
result_p = pd.DataFrame(res['x'].reshape(2, 4))#, columns=qjkl.columns, index = qjkl.index)
result_p/= result_p.sum().sum()

In [570]:
qjkl

CODUSU                                                              \
IV6         1                                       2                       
IV3         1         2         3         4         1         2         3   
IV5                                                                         
1    0.810820  0.052152  0.001005  0.002011  0.003445  0.004332  0.000177   
2    0.062562  0.045055  0.001257  0.000311  0.002410  0.010646  0.001627   

                                                       
IV6                   3                                
IV3         4         1         2         3         4  
IV5                                                    
1    0.000030  0.000266  0.000237  0.000412  0.000087  
2    0.000059  0.000163  0.000680  0.000222  0.000033

In [573]:
result_p_st = result_p.stack().stack().reset_index()

result_p_st.groupby('IV5')['CODUSU'].sum()

,0,1,2,3
0,0.777901,0.058902,0.010430,0.011383
1,0.068768,0.052176,0.010668,0.009771


In [516]:
result_p_st.groupby('IV3')['CODUSU'].sum()

IV3
1    0.716227
2    0.197093
3    0.051049
4    0.035632
Name: CODUSU, dtype: float64

In [517]:
result_p_st.groupby('IV6')['CODUSU'].sum()

IV6
1    0.827886
2    0.113528
3    0.058586
Name: CODUSU, dtype: float64

In [604]:
M_0 = qjkl['CODUSU'][1].round(2).values + 0.01
marg = np.array([ 0.17928839,  0.30580031,  0.09560977,  0.3615159 ,  0.02412938,
        0.00890009,  0.01495176,  0.0098044 ])
ix = [[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31]]

constraints = []
for j in range(8):
    def f(M, ix = ix, marg = marg, j = j):
        return M[ix[j]].sum() - marg[j]

    constraints.append(f)
    
cons = [{'type':'eq', 'fun': con} for con in constraints]

def MaxEnt(p_mat, q_mat = M_0):
    p_mat = np.array(p_mat).reshape(q_mat.shape)
    return (p_mat*np.log(p_mat/q_mat)).sum()
    
res = minimize(lambda M: MaxEnt(M), M_0, constraints = cons)
res['x'].round(3).reshape(M_0.shape)

/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:24: RuntimeWarning: invalid value encountered in log


array([[ 0.157,  0.011,  0.006,  0.006],
       [ 0.249,  0.034,  0.011,  0.011],
       [ 0.07 ,  0.009,  0.009,  0.009],
       [ 0.274,  0.066,  0.011,  0.011],
       [ 0.01 ,  0.005,  0.005,  0.005],
       [ 0.002,  0.002,  0.002,  0.002],
       [ 0.004,  0.004,  0.004,  0.004],
       [ 0.007,  0.001,  0.001,  0.001]])

In [611]:
print res['x'].reshape(M_0.shape).sum(axis = 1)
print res['x'].reshape(M_0.shape).sum(axis = 0)

[ 0.17928839  0.30580031  0.09560977  0.3615159   0.02412938  0.00890009
  0.01495176  0.0098044 ]
[ 0.77193675  0.13154095  0.04827543  0.04824687]


In [605]:
qjkl['CODUSU'][1].round(2).values

array([[ 0.27,  0.01,  0.  ,  0.  ],
       [ 0.21,  0.02,  0.  ,  0.  ],
       [ 0.07,  0.  ,  0.  ,  0.  ],
       [ 0.24,  0.05,  0.  ,  0.  ],
       [ 0.01,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.07,  0.  ,  0.  ,  0.  ]])

In [398]:
np.array([[ 0.157,  0.011,  0.006,  0.006],
       [ 0.249,  0.034,  0.011,  0.011],
       [ 0.07 ,  0.009,  0.009,  0.009],
       [ 0.274,  0.066,  0.011,  0.011],
       [ 0.01 ,  0.005,  0.005,  0.005],
       [ 0.002,  0.002,  0.002,  0.002],
       [ 0.004,  0.004,  0.004,  0.004],
       [ 0.007,  0.001,  0.001,  0.001]]).sum(axis  =1)

([ 0.17928839,  0.30580031,  0.09560977,  0.3615159 ,  0.02412938,
        0.00890009,  0.01495176,  0.0098044 ])

array([ 0.18 ,  0.305,  0.097,  0.362,  0.025,  0.008,  0.016,  0.01 ])

In [345]:
M_0 = np.array([[1, 0.1], [0.1, 1]])

constraints = []
for i in [0, 1]:
    def f(M, ix = ix, marg = marg, i = i):
        return M[ix[i]].sum() - marg[i]

    constraints.append(f)
    
cons = [{'type':'eq', 'fun': con} for con in constraints]

def MaxEnt(p_mat, q_mat = M_0):
    p_mat = np.array(p_mat).reshape(q_mat.shape)
    return (p_mat*np.log(p_mat/q_mat)).sum()
    
res = minimize(lambda M: MaxEnt(M), M_0, constraints = cons)
res['x'].round(2).reshape(M_0.shape)

/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: RuntimeWarning: invalid value encountered in log


array([[ 0.27,  0.03],
       [ 0.06,  0.64]])

In [335]:
M_0

array([[ 1. ,  0.1],
       [ 0.1,  1. ]])

In [264]:
from scipy.optimize import minimize
p_0 = q_mat.flatten()
res = minimize(lambda p: MaxEnt(p, q_mat), p_0, constraints = {'type':'eq', 'fun': f})
res['x']

/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()


array([  2.09754849e-01,   1.28741285e-02,   1.25216252e-03,
         8.53911549e-04,   5.57725191e-04,   7.59823300e-04,
         3.85516512e-05,   2.85302347e-04,   6.78548664e-06,
         5.80962004e-06,   4.78594649e-05,   5.65118931e-06,
         1.62202731e-01,   1.52433604e-02,   1.24559200e-04,
         2.27937691e-04,   4.46327927e-04,   7.14422172e-04,
         5.14872019e-05,   1.05571447e-04,   2.52575476e-05,
         1.32705092e-04,   1.40739768e-05,   5.80474848e-05,
         6.39401203e-02,   3.17060616e-03,   1.79186862e-05,
         1.00260016e-05,   2.08108399e-05,   1.17536930e-04,
         2.37065770e-04,   7.33754460e-05,   1.95948894e-03,
         7.26899000e-05,   9.95382222e-06,   6.51507140e-06,
         1.90489370e-01,   4.81708429e-02,   7.24254893e-04,
         2.27937691e-04,   4.02750006e-04,   6.41580750e-03,
         4.20581685e-04,   5.20637281e-06,   3.40221122e-04,
         4.71758458e-04,   7.01878927e-05,   2.28394137e-05,
         7.25023228e-03,

In [288]:
# f(res['x'], ix = ix)
marcond
# res['x'][ix].sum()

0.0054837974231155533

In [437]:
pd.DataFrame(res['x'].reshape(8, 12), columns = qjkl.columns).stack().stack().reset_index().groupby('IV3')['CODUSU'].sum()

IV3
1    0.827061
2    0.156737
3    0.011945
4    0.004256
Name: CODUSU, dtype: float64

In [435]:
margins[dim]

1 Por cañeria dentro de la vivienda               0.885473
2 Fuera de la vivienda pero dentro del terreno    0.092493
3 Fuera del terreno                               0.022033
dtype: float64

In [438]:
qjkl

CODUSU                                                              \
IV6         1                                       2                       
IV3         1         2         3         4         1         2         3   
IV4                                                                         
1    0.265412  0.012811  0.000811  0.000708  0.000855  0.001120  0.000059   
2    0.207639  0.021671  0.000295  0.000428  0.001459  0.002329  0.000133   
3    0.069758  0.002020  0.000074  0.000118  0.000074  0.000133  0.000229   
4    0.244096  0.054707  0.000737  0.000428  0.002742  0.010231  0.000899   
5    0.009862  0.002226  0.000059  0.000177  0.000221  0.000354  0.000177   
6    0.000516  0.000486  0.000088  0.000029  0.000029  0.000251  0.000206   
7    0.003096  0.002565  0.000177  0.000029  0.000118  0.000428  0.000324   
9    0.070363  0.000428  0.000015  0.000398  0.000339  0.000088  0.000023   

                                                       
IV6                   3                                
IV3         4         1         2         3         4  
IV4                                                    
1    0.000091  0.000044  0.000015  0.000102  0.000022  
2    0.000155  0.000059  0.000236  0.000015  0.000037  
3    0.000048  0.001613  0.000427  0.000054  0.000012  
4    0.000044  0.000310  0.000604  0.000103  0.000044  
5    0.000015  0.000015  0.000029  0.000029  0.000003  
6    0.000015  0.000150  0.000029  0.000044  0.000001  
7    0.000015  0.000252  0.000067  0.000029  0.000002  
9    0.000005  0.000165  0.000044  0.000006  0.000001

In [439]:
pd.DataFrame(res['x'].reshape(8, 12), columns = qjkl.columns)

CODUSU                                                              \
IV6         1                                       2                       
IV3         1         2         3         4         1         2         3   
0    0.239420  0.012570  0.000812  0.000554  0.003182  0.004093  0.000222   
1    0.188982  0.019796  0.000348  0.000307  0.005758  0.009386  0.000478   
2    0.063324  0.001807  0.000063  0.000115  0.000312  0.000551  0.000833   
3    0.222935  0.051044  0.000761  0.000451  0.010801  0.041484  0.003333   
4    0.009069  0.001908  0.000046  0.000204  0.001062  0.001634  0.000694   
5    0.000449  0.000482  0.000074  0.000023  0.000127  0.000959  0.000862   
6    0.002786  0.002248  0.000163  0.000030  0.000459  0.001715  0.001085   
7    0.063863  0.000436  0.000015  0.000388  0.001476  0.000356  0.000086   

                                                       
IV6                   3                                
IV3         4         1         2         3         4  
0    0.000380  0.000271  0.000067  0.000471  0.000096  
1    0.000633  0.000257  0.001330  0.000085  0.000179  
2    0.000170  0.007854  0.001026  0.000306  0.000066  
3    0.000161  0.001747  0.002977  0.000559  0.000214  
4    0.000054  0.000069  0.000169  0.000176  0.000059  
5    0.000055  0.000788  0.000152  0.000216  0.000010  
6    0.000057  0.001171  0.000323  0.000179  0.000012  
7    0.000035  0.000898  0.000226  0.000077  0.000003

### Lagrangian function

In [427]:
# L(x0, q_mat, margins)

In [529]:
def L(x, q_mat, margins):
    shape = np.array(q_mat.shape)
    len_a = shape.sum()
    p_mat = np.array(x[:-len_a]).reshape(shape)
    a = np.array(x[-len_a:])
    
    if np.random.rand() < 0.0001:
        print p_mat
        print a

    MaxEnt = (p_mat*np.log(p_mat/q_mat))
    
    a_ix = np.insert(shape.cumsum(), [0], 0)
    
    lambda_terms = np.zeros(len(shape))
    for dim in range(len(shape)):
        lambda_terms[dim] = np.dot(a[a_ix[dim]:a_ix[dim + 1]], 
                                   margins[dim].values - p_mat.sum(axis = tuple([i for i in range(len(shape)) if i != dim])))

    return MaxEnt.sum() + lambda_terms.sum()

In [ ]:
margins[dim].values - p_mat.sum(axis = tuple([i for i in range(len(shape)) if i != dim]))

In [540]:
margins[dim].values - p_mat.sum(axis = tuple([i for i in range(len(shape)) if i != dim]))

array([ 0.05428839,  0.18080031, -0.02939023,  0.2365159 , -0.10087062,
       -0.11609991, -0.11004824, -0.1151956 ])

In [549]:
constraints = []
for dim in range(len(shape)):
    def f(p_mat, dim = dim):
        return p_mat.sum(axis = tuple([t for t in range(len(shape)) if t != dim])) - margins[dim].values
    constraints.append(f)

In [550]:
constraints

[<function __main__.f>, <function __main__.f>, <function __main__.f>]

In [165]:
shape = np.array(q_mat.shape)
len_a = shape.sum()
p_mat = np.array(x[:-len_a]).reshape(shape)
    
def MaxEnt(p_mat, q_mat):
    p_mat = np.array(p_mat).reshape(q_mat.shape)
    return (p_mat*np.log(p_mat/q_mat)).sum()

NameError: name 'x' is not defined

In [291]:
ix = cube_tr[i].flatten()

def f(p, ix = ix):
    return p[ix].sum() - .1

con = [{'type':'eq', 'fun': f}]
con

[{'fun': <function __main__.f>, 'type': 'eq'}]

In [212]:
# def make_f(kwargs, k):
#     def f():
#         print(k, kwargs[k])
#     return f

# for i in [0, 1]:
#     f = make_f(cube_tr[i].flatten(), cube.flatten())

In [183]:
constraints = []
for i in [0, 1]:
    def f(p_, shape = shape, i = i):
#         p_mat = p_.reshape(shape)
        
        # There should be a translator of the indices from cube jkl to plain index.
        # For each axis, fix the axis, then do a for loop in which the L (or J, K) equations are written as functions
        # This should be done in paper first
        
#         return p_mat.sum(axis = tuple([t for t in range(len(shape)) if t != dim])) - margins[dim].values

        return M[i] + M[3] - .5
    constraints.append(f)
    
cons = [{'type':'eq', 'fun': con} for con in constraints]

In [328]:
ix = [[0, 2], [1, 3]]

In [330]:
M_0 = np.array([[1, 0], [0, 1]])

constraints = []
for i in [0, 1]:
    def f(M, i = i):
#         return M[1] + M[3] - .5
        return M[ix[i]].sum() - i

    constraints.append(f)
    
cons = [{'type':'eq', 'fun': con} for con in constraints]

def Test(M):
    return (M**2).sum()
    
res = minimize(lambda M: Test(M), M_0, constraints = cons)
res['x'].round(2).reshape(M_0.shape)

array([[ 0. ,  0.5],
       [-0. ,  0.5]])

In [325]:
# ix[i]
# M_0[ix[i]]


IndexError: list index out of range

In [599]:
p_0 = uncorr_fill_p.values.reshape(shape)

res = minimize(lambda p_mat: MaxEnt(p_mat, q_mat), p_0,# constraints = constraints,
                   method='Nelder-Mead',
                  options={'disp': True, 
                           'maxiter': 10000000, 
                           'xatol': 0.00001, 
                           'fatol': 0.00001, 
                           'maxfev': 1000000})

         Current function value: nan
         Iterations: 1
         Function evaluations: 10976
         Gradient evaluations: 112


/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: OptimizeWarning: Unknown solver options: fatol, xatol, maxfev
/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: RuntimeWarning: invalid value encountered in log


In [600]:
p_mat_ = np.array(res['x']).reshape(shape)
p_= pd.DataFrame(p_mat_.reshape(qjkl.shape), 
             columns = qjkl.columns, index = qjkl.index)
p_ = p_/p_.sum().sum()


In [601]:
p_

CODUSU                                                              \
IV6         1                                       2                       
IV3         1         2         3         4         1         2         3   
IV4                                                                         
1    0.001284  0.011385  0.015534  0.007155  0.021915  0.012441  0.017681   
2    0.005892  0.011431  0.024693  0.013305  0.021910  0.011268  0.016039   
3    0.005484  0.018609  0.026019  0.014047  0.032716  0.021359  0.005927   
4    0.005920  0.006934  0.020255  0.014297  0.019164  0.003490  0.005690   
5    0.008925  0.009874  0.019182  0.003485  0.018046  0.007387 -0.000715   
6    0.020501  0.012977  0.010869  0.008194  0.024076  0.003520 -0.007538   
7    0.012956  0.006198  0.009835  0.011268  0.018935  0.003429 -0.007143   
9   -0.008056  0.014316  0.022061 -0.006658  0.010175  0.010266  0.005928   

                                                       
IV6                   3                                
IV3         4         1         2         3         4  
IV4                                                    
1    0.005927  0.030968  0.029605  0.005927  0.005928  
2    0.005927  0.032427  0.016338  0.020559  0.005928  
3    0.005927  0.005926  0.005926  0.005927  0.005930  
4    0.014355  0.023591  0.011753  0.010019  0.005927  
5    0.004827  0.025593  0.013612  0.001404  0.005941  
6   -0.001077  0.005927  0.007702 -0.006904  0.005967  
7    0.001993  0.005927  0.005927 -0.001430  0.005951  
9    0.005935  0.005927  0.005927  0.005934  0.005963

In [587]:
qjkl

CODUSU                                                              \
IV6         1                                       2                       
IV3         1         2         3         4         1         2         3   
IV4                                                                         
1    0.265412  0.012811  0.000811  0.000708  0.000855  0.001120  0.000059   
2    0.207639  0.021671  0.000295  0.000428  0.001459  0.002329  0.000133   
3    0.069758  0.002020  0.000074  0.000118  0.000074  0.000133  0.000229   
4    0.244096  0.054707  0.000737  0.000428  0.002742  0.010231  0.000899   
5    0.009862  0.002226  0.000059  0.000177  0.000221  0.000354  0.000177   
6    0.000516  0.000486  0.000088  0.000029  0.000029  0.000251  0.000206   
7    0.003096  0.002565  0.000177  0.000029  0.000118  0.000428  0.000324   
9    0.070363  0.000428  0.000015  0.000398  0.000339  0.000088  0.000023   

                                                       
IV6                   3                                
IV3         4         1         2         3         4  
IV4                                                    
1    0.000091  0.000044  0.000015  0.000102  0.000022  
2    0.000155  0.000059  0.000236  0.000015  0.000037  
3    0.000048  0.001613  0.000427  0.000054  0.000012  
4    0.000044  0.000310  0.000604  0.000103  0.000044  
5    0.000015  0.000015  0.000029  0.000029  0.000003  
6    0.000015  0.000150  0.000029  0.000044  0.000001  
7    0.000015  0.000252  0.000067  0.000029  0.000002  
9    0.000005  0.000165  0.000044  0.000006  0.000001

In [602]:
p_stacked = p_.stack().stack().reset_index()
print p_stacked.groupby('IV4')['CODUSU'].sum()
print p_stacked.groupby('IV6')['CODUSU'].sum()
print p_stacked.groupby('IV3')['CODUSU'].sum()

IV4
1    0.165750
2    0.185716
3    0.153799
4    0.141395
5    0.117563
6    0.084213
7    0.073845
9    0.077720
Name: CODUSU, dtype: float64
IV6
1    0.358172
2    0.319779
3    0.322049
Name: CODUSU, dtype: float64
IV3
1    0.356126
2    0.261677
3    0.225754
4    0.156443
Name: CODUSU, dtype: float64


In [589]:
margins

[1 Membrana / cubierta asfáltica                0.179288
 2 Baldosa / losa sin cubierta                  0.305800
 3 Pizarra / teja                               0.095610
 4 Chapa de metal sin cubierta                  0.361516
 5 Chapa de fibrocemento / plástico             0.024129
 6 Chapa de cartón                              0.008900
 7 Caña / tabla / paja con barro / paja sola    0.014952
 9 N/S. Depto en propiedad horizontal           0.009804
 dtype: float64, 1 Por cañeria dentro de la vivienda               0.885473
 2 Fuera de la vivienda pero dentro del terreno    0.092493
 3 Fuera del terreno                               0.022033
 dtype: float64, 1 Mosaico / baldosa / madera / cerámica / alfombra    0.765829
 2 Cemento / ladrillo fijo                             0.202845
 3 Ladrillo suelto / tierra                            0.025842
 4 Otro                                                0.005484
 dtype: float64]

In [580]:
# from scipy.optimize import minimize


# shape = [len(m) for m in margins]
# q_mat = qjkl.values.reshape(shape)

# def pT(q_mat, margins):
#     # This generates a left-stochastic matrix
    
# #     p_0 = np.ones(np.array(q_mat.shape))/len(qjkl.values.flatten())
#     p_0 = uncorr_fill_p.values.reshape(shape)
#     a_ = np.ones(np.array(q_mat.shape).sum())
# #     a_ = np.array([ 3.82209837, -0.67239368,0.9138076,1.71378012,1.70591917,1.34566778,
# #  -0.10336548,1.71989107,2.7796474,-1.3430727,1.78366337,4.72804659,
# #  -0.67614996,3.08805061,2.84402776])
#     x0 = list(p_0.flatten()) + list(a_)

#     res = minimize(lambda x: L(x, q_mat, margins), x0, 
#                    method='Nelder-Mead',
#                   options={'disp': True, 
#                            'maxiter': 10000000, 
#                            'xatol': 0.00001, 
#                            'fatol': 0.00001, 
#                            'maxfev': 1000000})
   
#     return res

In [581]:
# res = pT(q_mat, margins)

In [582]:
p_mat_ = np.array(res['x'][:-len_a]).reshape(shape)
a_ = np.array(res['x'][-len_a:])

ValueError: total size of new array must be unchanged

In [538]:
p_= pd.DataFrame(p_mat_.reshape(qjkl.shape), 
             columns = qjkl.columns, index = qjkl.index)
p_ = p_/p_.sum().sum()
# p_.sum().sum()

2.8183442948302186

In [534]:
p_

CODUSU                                                                \
IV6         1                                               2                 
IV3         1         2         3             4             1             2   
IV4                                                                           
1    0.141882  0.015641  0.001328  2.952166e-04  2.900262e-03  1.369149e-03   
2    0.262956  0.032421  0.002498  2.435911e-04  1.394959e-03  1.051611e-03   
3    0.065675  0.000549  0.001293  3.456847e-06  6.175743e-07  9.337652e-04   
4    0.385724  0.041670  0.002848  1.346926e-03  6.949308e-03  3.386692e-09   
5    0.008833  0.001651  0.000043  1.973323e-05  4.050011e-04  1.286109e-04   
6    0.004151  0.000546  0.000015  3.406675e-08  7.559034e-06  1.126271e-09   
7    0.001333  0.000590  0.000001  2.572921e-05  1.244644e-03  8.299050e-05   
9    0.002423  0.000062  0.000093  5.341664e-06  1.821901e-04  2.266339e-05   

                                                                             
IV6                                 3                                        
IV3         3             4         1         2             3             4  
IV4                                                                          
1    0.000201  6.794651e-05  0.000841  0.000366  2.796933e-08  4.640061e-07  
2    0.000057  2.288506e-06  0.000751  0.000052  8.421282e-07  2.810720e-11  
3    0.000051  2.070349e-05  0.000081  0.000052  3.431275e-05  3.457732e-06  
4    0.000414  1.197275e-04  0.002830  0.000415  1.396692e-04  7.946080e-06  
5    0.000030  4.313255e-06  0.000238  0.000033  2.232306e-06  2.002925e-06  
6    0.000011  7.460832e-07  0.000114  0.000054  1.049171e-11  4.735263e-07  
7    0.000013  2.638256e-06  0.000045  0.000010  3.784740e-06  1.352610e-06  
9    0.000009  3.443087e-06  0.000071  0.000002  5.167761e-08  8.386387e-10

In [535]:
qjkl

CODUSU                                                              \
IV6         1                                       2                       
IV3         1         2         3         4         1         2         3   
IV4                                                                         
1    0.265412  0.012811  0.000811  0.000708  0.000855  0.001120  0.000059   
2    0.207639  0.021671  0.000295  0.000428  0.001459  0.002329  0.000133   
3    0.069758  0.002020  0.000074  0.000118  0.000074  0.000133  0.000229   
4    0.244096  0.054707  0.000737  0.000428  0.002742  0.010231  0.000899   
5    0.009862  0.002226  0.000059  0.000177  0.000221  0.000354  0.000177   
6    0.000516  0.000486  0.000088  0.000029  0.000029  0.000251  0.000206   
7    0.003096  0.002565  0.000177  0.000029  0.000118  0.000428  0.000324   
9    0.070363  0.000428  0.000015  0.000398  0.000339  0.000088  0.000023   

                                                       
IV6                   3                                
IV3         4         1         2         3         4  
IV4                                                    
1    0.000091  0.000044  0.000015  0.000102  0.000022  
2    0.000155  0.000059  0.000236  0.000015  0.000037  
3    0.000048  0.001613  0.000427  0.000054  0.000012  
4    0.000044  0.000310  0.000604  0.000103  0.000044  
5    0.000015  0.000015  0.000029  0.000029  0.000003  
6    0.000015  0.000150  0.000029  0.000044  0.000001  
7    0.000015  0.000252  0.000067  0.000029  0.000002  
9    0.000005  0.000165  0.000044  0.000006  0.000001

In [536]:
p_stacked = p_.stack().stack().reset_index()
print p_stacked.groupby('IV4')['CODUSU'].sum()
print p_stacked.groupby('IV3')['CODUSU'].sum()
print p_stacked.groupby('IV6')['CODUSU'].sum()

IV4
1    0.164892
2    0.301428
3    0.068698
4    0.442466
5    0.011390
6    0.004899
7    0.003354
9    0.002874
Name: CODUSU, dtype: float64
IV3
1    0.891033
2    0.097703
3    0.009087
4    0.002178
Name: CODUSU, dtype: float64
IV6
1    0.976167
2    0.017681
3    0.006152
Name: CODUSU, dtype: float64


In [527]:
margins

[1 Membrana / cubierta asfáltica                0.179288
 2 Baldosa / losa sin cubierta                  0.305800
 3 Pizarra / teja                               0.095610
 4 Chapa de metal sin cubierta                  0.361516
 5 Chapa de fibrocemento / plástico             0.024129
 6 Chapa de cartón                              0.008900
 7 Caña / tabla / paja con barro / paja sola    0.014952
 9 N/S. Depto en propiedad horizontal           0.009804
 dtype: float64, 1 Por cañeria dentro de la vivienda               0.885473
 2 Fuera de la vivienda pero dentro del terreno    0.092493
 3 Fuera del terreno                               0.022033
 dtype: float64, 1 Mosaico / baldosa / madera / cerámica / alfombra    0.765829
 2 Cemento / ladrillo fijo                             0.202845
 3 Ladrillo suelto / tierra                            0.025842
 4 Otro                                                0.005484
 dtype: float64]

In [528]:
q_stacked = qjkl.stack().stack()
print q_stacked.groupby('IV4')['CODUSU'].sum()
print q_stacked.groupby('IV3')['CODUSU'].sum()
print q_stacked.groupby('IV6')['CODUSU'].sum()

IV4
1    0.282050
2    0.234455
3    0.074560
4    0.314945
5    0.013167
6    0.001847
7    0.007102
9    0.071875
Name: CODUSU, dtype: float64
IV3
1    0.879189
2    0.113298
3    0.004689
4    0.002823
Name: CODUSU, dtype: float64
IV6
1    0.972226
2    0.023210
3    0.004565
Name: CODUSU, dtype: float64


In [11]:
## Model?
# It would be a max ent prolem minimizing distance between pjkl and priors qjkl, 
# using lagrange multipliers with the conditions that (hyper-) row and column sums match the margins.

In [12]:
# import scipy.maxentropy.conditionalmodel as cm

In [13]:
# def f_1(x):
#     return x

# F = [f_1]
# samplespace = np.ones((8, 4, 4))
# K = qjkl.values.reshape(8, 4, 4)
# cm(F, samplespace, K)